In [1]:
#Run the below line of code in the annaconda prompt
#conda install quandl
#%reset
from pandas_datareader import data

import matplotlib.pyplot as plt
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import quandl
import numpy as np
import csv
import datetime
import time
from datetime import datetime
from datetime import timedelta
import dill

####download daily stock data from yahoo finance and save in "raw data" folder
####in the working directory specified below

###set working directory
os.chdir(r'C:\Users\Jeff\Box Sync\Kumar Lab\Writing\NAMS Paper\Data\Zoom Data\Oral Sessions')
path_raw=os.getcwd() +'\\raw_data'

#create list of raw security data files from "raw_data" folder
files = [f for f in listdir(path_raw) if isfile(join(path_raw,f))]
#files = list(['UserQos_81116913656.csv'])

#import webinar table and format meeting ID
webinar_table=pd.read_csv(os.getcwd() + "\\raw_data\\webinars_overview.csv")
webinar_table['Meeting IDa'] = webinar_table['Meeting ID'].str[0:3]
webinar_table['Meeting IDb'] = webinar_table['Meeting ID'].str[4:8]
webinar_table['Meeting IDc'] = webinar_table['Meeting ID'].str[9:]
webinar_table['Meeting ID final']=webinar_table['Meeting IDa']+webinar_table['Meeting IDb']+webinar_table['Meeting IDc']
webinar_table=webinar_table.drop(columns=['Meeting ID','Meeting IDa','Meeting IDb','Meeting IDc','Host','Email','User Type','Department','Group','Start Time','End Time','Duration (hh:mm:ss)','Participants','Phone','VoIP','3rd Party Audio','Video','Screen Sharing','Recording','CRC','Encryption'])
webinar_table=webinar_table.rename(columns={"Meeting ID final":"Meeting ID"})
webinar_table['Stamp Number']=0

columns = ['Participant','Device','IP Address','Location','Network Type','Microphone','Speaker','Camera','Data Center','Connection Type','Join Time','Leave Time','Version','Audio (Receiving) Bitrate','Audio (Sending) Bitrate','Audio (Receiving) Latency','Audio (Sending) Latency','Audio (Receiving) Jitter','Audio (Sending) Jitter','Audio (Receiving) Packet Loss-Avg(Max)','Audio (Sending) Packet Loss-Avg(Max)','Video (Receiving) Bitrate','Video (Sending) Bitrate','Video (Receiving) Latency','Video (Sending) Latency','Video (Receiving) Jitter','Video (Sending) Jitter','Video (Receiving) Packet Loss-Avg(Max)','Video (Sending) Packet Loss-Avg(Max)','Video (Receiving) Resolution','Video (Sending) Resolution','Video (Receiving) Frame Rate','Video (Sending) Frame Rate','Screen Sharing (Receiving) Bitrate','Screen Sharing (Sending) Bitrate','Screen Sharing (Receiving) Latency','Screen Sharing (Sending) Latency','Screen Sharing (Receiving) Jitter','Screen Sharing (Sending) Jitter','Screen Sharing (Receiving) Packet Loss-Avg(Max)','Screen Sharing (Sending) Packet Loss-Avg(Max)','Screen Sharing (Receiving) Resolution','Screen Sharing (Sending) Resolution','Screen Sharing (Receiving) Frame Rate','Screen Sharing (Sending) Frame Rate','Zoom Min Cpu Usage','Zoom Avg Cpu Usage','Zoom Max Cpu Usage','System Max Cpu Usage']

webinar_data_all = pd.DataFrame()

for f in files:
    filename=f[8:-4]
    print(filename)
    
    data=pd.read_csv(path_raw + "\\" + f,names=columns)
    #Get date of webinar
    date = datetime.strptime(data.iloc[1,7], "%b %d, %Y %I:%M %p").date()
    #remove admin data from first rows
    data=data.iloc[3:,:]

    #define and remove unnecessary columns
    columns2 = ['Device','Network Type','Microphone','Speaker','Camera','Data Center','Connection Type','Version','Audio (Receiving) Bitrate','Audio (Sending) Bitrate','Audio (Receiving) Latency','Audio (Sending) Latency','Audio (Receiving) Jitter','Audio (Sending) Jitter','Audio (Receiving) Packet Loss-Avg(Max)','Audio (Sending) Packet Loss-Avg(Max)','Video (Receiving) Bitrate','Video (Sending) Bitrate','Video (Receiving) Latency','Video (Sending) Latency','Video (Receiving) Jitter','Video (Sending) Jitter','Video (Receiving) Packet Loss-Avg(Max)','Video (Sending) Packet Loss-Avg(Max)','Video (Receiving) Resolution','Video (Sending) Resolution','Video (Receiving) Frame Rate','Video (Sending) Frame Rate','Screen Sharing (Receiving) Bitrate','Screen Sharing (Sending) Bitrate','Screen Sharing (Receiving) Latency','Screen Sharing (Sending) Latency','Screen Sharing (Receiving) Jitter','Screen Sharing (Sending) Jitter','Screen Sharing (Receiving) Packet Loss-Avg(Max)','Screen Sharing (Sending) Packet Loss-Avg(Max)','Screen Sharing (Receiving) Resolution','Screen Sharing (Sending) Resolution','Screen Sharing (Receiving) Frame Rate','Screen Sharing (Sending) Frame Rate','Zoom Min Cpu Usage','Zoom Avg Cpu Usage','Zoom Max Cpu Usage','System Max Cpu Usage']
    data=data.drop(columns = columns2)
    #parse timestamp data
    data['Leave Time2']=data['Leave Time'].str[:8]
    data['Leave Reason']=data['Leave Time'].str[9:]
    data=data.drop(columns=['Leave Time'])
    data=data.rename(columns={"Leave Time2":"Leave Time"})

    #convert timestamp strings to datetime format
    for row in data.index:
            if isinstance(data.loc[row,'Join Time'],str):
                data.loc[row,'Join Time'] = datetime.strptime(data.loc[row,'Join Time'], "%I:%M %p")
                data.loc[row,'Join Time']=data.loc[row,'Join Time'].replace(year=date.year,month=date.month,day=date.day)
            if isinstance(data.loc[row,'Leave Time'],str):
                data.loc[row,'Leave Time'] = datetime.strptime(data.loc[row,'Leave Time'], "%I:%M %p")
                data.loc[row,'Leave Time']=data.loc[row,'Leave Time'].replace(year=date.year,month=date.month,day=date.day)

    #define latest recorded leave stamp
    stamps = data['Leave Time']
    stamps = stamps.dropna()
    endtime = max(stamps)

    for row in data.index:
        #replace missing leave stamp with latest recorded stamp
        if pd.isnull(data.loc[row,'Leave Time']):
            data.loc[row,'Leave Time']=endtime
        #replace missing leave reason with "unknown"
        if pd.isnull(data.loc[row,'Leave Reason']):
            data.loc[row,'Leave Reason']="Unknown"
        #remove name from leave reason column
        else:
            data.loc[row,'Leave Reason']=data.loc[row,'Leave Reason'][len(data.loc[row,'Participant'])+1:-1]
    unk = data.loc[data['Leave Reason']=="Unknown"]
    unk_ind = list(unk.index)
    data = data.drop(unk_ind)
            
    session=webinar_table.loc[webinar_table['Meeting ID']== filename]
    session_name=session.iloc[0,0]
    session_room=session.iloc[0,1]
    session_index=session.index
    ###produce directory for script outputs
    output_path=os.getcwd() + '\\clean_data\\'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    webinar_table.loc[session_index,'Stamp Number'] = len(data.index)
    data['Session']=session_name
    data['Room']=session_room
    
    webinar_data_all=webinar_data_all.append(data)
    webinar_data_all.index = range(len(webinar_data_all.index))
    data.to_csv(output_path + session_name + '.csv')

webinar_table.to_csv(output_path + 'clean_data_summary.csv')
webinar_data_all.to_csv(output_path + 'webinar_data_all.csv')

dill.dump_session('clean_webinar_data.db')

81116913656
81210740740
81225697194
81349150301
81388491271
82274636203
82302039156
82415416364
82863003768
82972923069
83026554952
83083006574
83715378725
84063816459
84197510572
84536200249
84551022759
84893350964
84945149697
85041293914
85086098478
85179317802
85753118004
85779420010
85976194075
85992203761
86384155374
87064742151
87689506643
88385803393
88568042456
89118189203
89642598858
